# EMBO Practical Course "Advanced methods in bioimage analysis"

***

Homepage: https://www.embl.org/about/info/course-and-conference-office/events/bia23-01/

***

## Day 2 - Session 1: Image Data Management - 11:30 to 12:30 "GO!"

<table style="table { position: relative;  display: inline-block; } img {  position: absolute;  left: 0;  right: 0;  width: auto;  height: 100%;  object-fit: cover;  object-position: center;}">
    <tr>
        <td style="vertical-align: top">
            <h3>Introduction</h3>
            <p>
                In this notebook, we'll switch from looking at the filesystem to looking at the cloud.
                "S3" and other object storage backends get rid of many of the POSIX constraints, leaving
                many tools unable to (⚠ Caveat: without something like the mounting tool). After looking
                differences and the available tools, we'll see how to migrate to OME-Zarr and what that
                gives you access to.
            </p>
            <p>
                Outline:
                <ol start="5">
                    <li>The Cloud
                        <ol type="a">
                            <li>S3</li>
                            <li>aws &amp; s3</li>
                            <li>Our bucket</li>
                            <li>Extras: time permitting
                            </li>
                        </ol>
                    </li>
                        <li>OME-Zarr
                        <ol type="a">
                            <li>Generating OME-Zarrs</li>
                            <li>Publishing OME-Zarrs</li>
                            <li>Looking deeper into OME-Zarrs</li>
                            <li>Other options</li>
                            <li>Time-permitting</li>
                        </ol>
                    </li>
                </ol>
            </p>
        </td>
        <td>
            <center>
                <img src="images/falk/adam-uploads-300dpi.png"/>
                <small>
                    <a href="https://github.com/zarr-developers/zarr-illustrations-falk-2022#adam-uploads">"Adam uploads"</a>
                    by Henning Falk, ©2022 NumFOCUS, is used under a CC BY 4.0 license. Modifications to this photo include cropping.
                </small>
            </center>
        </td>
    </tr>
</table>

### Software versions used for this workshop:

* **bioformats2raw            0.7.0** ([install locally](https://github.com/glencoesoftware/bioformats2raw/releases/latest); requires Java)
* **minio-client                     2020.11.17** ([install locally](https://docs.min.io/docs/minio-client-complete-guide.html))
* **awscli                    1.29.30**
* **napari                    0.4.18**

In [1]:
##
## Setup & Sanity checks
##

YOURNAME=$(whoami)
WORKDIR=/scratch/${YOURNAME}/session1/
test -e ${WORKDIR} || {
    echo Please run the first the POSIX notebook first.
    exit 1
}
cd /scratch/${YOURNAME}/session1

## 5a. So what is the cloud?

From the [BAND user guide](https://docs.google.com/document/d/1TZBUsNIciGMH_g4aFj2Lu_upISxh5TV9FBMrvNDWmc8/edit#heading=h.4tth2eqswl6h):

> **Storage space**<br/>
> Disk space is shared and limited and although data stored in your home directory and under /scratch is persistent after you terminate your running desktop, we will remove content that is older than 3 months in both your home directory and /scratch.
Here are a few pointers:
> * If you know you’re not going to use BAND for a while, remove your data. 
> * If you can, use external storage such as S3 buckets or file sharing services.
> * If you have more than ~100 GB of data, put it under /scratch which has up to 20 TB in total.

### Quick motivating comparison

This won't work on your system because I haven't put 1GB files in each of your directories, but as an example:

```bash
$ ls -ltrad s3
lrwxrwxrwx 1 josh_openmicroscopy users   54 Aug 25 13:37 s3 -> /nfs/home/dea88c8ad47cb989/bioimagecourse2023/session1

$ ls -ltrad nfs
lrwxrwxrwx 1 josh_openmicroscopy users   36 Aug 25 13:37 nfs -> /scratch/bioimagecourse2023/session1

$ ls -ltrad local
drwxr-xr-x 2 josh_openmicroscopy users 4096 Aug 25 13:41 local
```

With the following script, I tested the speed of writing and then reading a 10MB file on each of these three directories:
```bash
set -e

SZ=${SZ:-10}
BIG=M
SMALL=K
SRC=urandom

PV="pv -s ${SZ}${BIG} -N"

for x in local nfs s3;
do
  dd if=/dev/$SRC bs=100${SMALL} count=${SZ}0 iflag=fullblock status=none | ${PV} "$x:w" > ${x}/${SZ}${BIG}B
done

for x in local nfs s3;
do
  cat ${x}/${SZ}${BIG}B |  $PV "$x:r" > /dev/null
  rm ${x}/${SZ}${BIG}B
done
```

The speeds show the differences that you need to be aware of:
```bash
  local:w: 9.77MiB 0:00:00 [ 139MiB/s] [========================> ] 97%            
    nfs:w: 9.77MiB 0:00:00 [25.0MiB/s] [========================> ] 97%            
     s3:w: 9.77MiB 0:00:00 [64.2MiB/s] [========================> ] 97%            
  local:r: 9.77MiB 0:00:00 [1.30GiB/s] [========================> ] 97%            
    nfs:r: 9.77MiB 0:00:00 [1.12GiB/s] [========================> ] 97%            
     s3:r: 9.77MiB 0:00:00 [ 131MiB/s] [========================> ] 97%
```
**P.S.** caching and similar can make these speeds vary *widely*.

**P.P.S** here "local" is really mounted as well. 😀

## 5b. Cloud tools

There are a number of tools that are built for working with the remote filesystems directly.

The first is the AWS cli from Amazon. It is likely the most complete tool, but not necessarily the easiest to use.

In [2]:
aws help | grep s3

       o s3
       o s3api
       o s3control
       o s3outposts


There are a number of different types of cloud storage and there are a number of tools that you can use to access your cloud storage, but here we're going to focus on a single one `mc` ("minio client"). [Minio](https://min.io/) is an implementation of S3 that you can host yourself (as EMBL does with https://s3.embl.de)

`mc` is provided by the minio project and is described as "a modern alternative to UNIX commands like ls, cat, cp, mirror, diff, find etc." The quickstart guide can be found under https://docs.minio.io/docs/minio-client-quickstart-guide.html For our purposes we'll focus on how to use it to upload and manage data in S3.

In [3]:
mc -h

NAME:
  mc - MinIO Client for cloud storage and filesystems.

USAGE:
  mc [FLAGS] COMMAND [COMMAND FLAGS | -h] [ARGUMENTS...]

COMMANDS:
  alias      set, remove and list aliases in configuration file
  ls         list buckets and objects
  mb         make a bucket
  rb         remove a bucket
  cp         copy objects
  mirror     synchronize object(s) to a remote site
  cat        display object contents
  head       display first 'n' lines of an object
  pipe       stream STDIN to an object
  share      generate URL for temporary access to an object
  find       search for objects
  sql        run sql queries on objects
  stat       show object metadata
  mv         move objects
  tree       list buckets and objects in a tree format
  du         summarize disk usage recursively
  retention  set retention for object(s)
  legalhold  manage legal hold for object(s)
  diff       list differences in object name, size, and date between two buckets
  rm         remove objects
  version    

#### Connections

The minio project provides a safe space for you to learn about S3: https://play.minio.io:9000/minio/ Here we've used the `mc` command to find the access information:

 * **"AccessKey"** is basically a user name.
 * **"SecretKey"** is basically a password. 
 * The URL is our **"endpoint"**, which differentiates it from the S3 servers provided by Amazon.

You can log in to the webpage and explore what the many other users have upload at https://play.minio.io:9000/minio/

The other two important concepts are:
 * **"buckets"** which is roughly like a shared namespace with permissions
 * and **"keys"** which will get to in a second.

In [4]:
mc config host list play

play
  URL       : https://play.min.io
  AccessKey : Q3AM3UQ867SPQQA43P2F
  SecretKey : zuf+tfteSlswRu7BJ86wekitnifILbZam1KYY3TG
  API       : S3v4
  Path      : auto




#### Using `mc` with a public S3 bucket 

In [5]:
mc config host add ebi https://uk1s3.embassy.ebi.ac.uk "" ""

Added `ebi` successfully.



In [6]:
mc ls ebi/idr/zarr/v0.4/

[2023-08-31 14:50:30 UTC]     0B idr0001A/
[2023-08-31 14:50:30 UTC]     0B idr0013A/
[2023-08-31 14:50:30 UTC]     0B idr0044A/
[2023-08-31 14:50:30 UTC]     0B idr0047A/
[2023-08-31 14:50:30 UTC]     0B idr0048A/
[2023-08-31 14:50:30 UTC]     0B idr0050A/
[2023-08-31 14:50:30 UTC]     0B idr0052A/
[2023-08-31 14:50:30 UTC]     0B idr0054A/
[2023-08-31 14:50:30 UTC]     0B idr0056B/
[2023-08-31 14:50:30 UTC]     0B idr0062A/
[2023-08-31 14:50:30 UTC]     0B idr0072B/
[2023-08-31 14:50:31 UTC]     0B idr0073A/
[2023-08-31 14:50:31 UTC]     0B idr0076A/
[2023-08-31 14:50:31 UTC]     0B idr0083A/
[2023-08-31 14:50:31 UTC]     0B idr0101A/
[2023-08-31 14:50:31 UTC]     0B idr0128E/
[2023-08-31 14:50:31 UTC]     0B idr0138A/



These are the files from the IDR which have been converted into OME-NGFF:

https://idr.github.io/ome-ngff-samples

<img src="images/samples.png"/>

### Exercise 4: configure an mc host named "embo"

In [7]:
echo mc config host list embo

mc config host list embo


## 5c. Our bucket

In [8]:
mc ls embo/

[2023-08-16 10:24:02 UTC]     0B bioimagecourse2023/



In [9]:
mc ls embo/bioimagecourse2023/

[2023-08-31 14:50:32 UTC]     0B .Trash-1566/
[2023-08-31 14:50:32 UTC]     0B session1/



<hr/>

## Half-time

<hr/>

## 6a. Generating OME-Zarrs

The two basic commands are `bioformats2raw` and `raw2ometiff`. Together they provide a pipeline to scalably convert large images into OME-TIFF. The primary caveat is that they require **twice** the storage for the conversion.


<center>
    <img src="images/blog-2019-12-converting-whole-slide-images.jpg" style="height:200px" />
    <small><a href="https://forum.image.sc/t/converting-whole-slide-images-to-ome-tiff-a-new-workflow/32110/4">Original 2019 post</a></small>
</center>

### bioformats2raw

In [10]:
# Remove previous runs
test -e a.ome.zarr && rm -rf a.ome.zarr || echo Not yet created

Not yet created


In [11]:
# Generate an OME-Zarr
bioformats2raw -p a.ome.tiff a.ome.zarr

OpenJDK 64-Bit Server VM warning: You have loaded library /tmp/opencv_openpnp11762032706019177913/nu/pattern/opencv/linux/x86_64/libopencv_java342.so which might have disabled stack guard. The VM will try to fix the stack guard now.
It's highly recommended that you fix the library with 'execstack -c <libfile>', or link it with '-z noexecstack'.
[0/0]   0% ?                                     ? 0/1 (0:00:00 / ?) 
[0/0] 100% ????????????????????????????????? 1/1 (0:00:00 / 0:00:00) [0/1]   0% ?                                     ? 0/1 (0:00:00 / ?) 


***

## 6b. Publishing your data with S3 ⚠️

You can then move the generated output to S3. **Note: this won't work unless you have a configured bucket that you can write to.**

In [12]:
# Upload it to 
mc cp -r a.ome.zarr/ embo/bioimagecourse2023/session1/${YOURNAME}/a.ome.zarr/

...A.ome.xml:  4.83 KiB / 4.83 KiB ┃▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓┃ 27.31 KiB/s 0s


Now you should be able to access the data on the web, by going to a URL like the following one:

https://hms-dbmi.github.io/vizarr/?source=https://s3.embl.de/bioimagecourse2023/session1/jamoore/a.ome.zarr/0

after updating your username whatever data you just uploaded, or the next cell should automatically open up a browser tab:

In [13]:
gio open https://hms-dbmi.github.io/vizarr/?source=https://s3.embl.de/bioimagecourse2023/session1/${YOURNAME}/a.ome.zarr/0

This viewer is [vizarr](https://github.com/hms-dbmi/vizarr) from the [Gehlenborg lab](http://gehlenborglab.org/) at Harvard Medical School. It can be accessed at https://hms-dbmi.github.io/vizarr for example to access data from the IDR: [link](http://hms-dbmi.github.io/vizarr/v0.1?source=https%3A%2F%2Fuk1s3.embassy.ebi.ac.uk%2Fidr%2Fzarr%2Fv0.1%2F6001240.zarr).

In [14]:
gio open https://ome.github.io/ome-ngff-validator/?source=https://s3.embl.de/bioimagecourse2023/session1/${YOURNAME}/a.ome.zarr


(firefox:51242): Gtk-WARNING **: 14:50:37.206: Locale not supported by C library.
	Using the fallback 'C' locale.


## 6c. Looking into OME-Zarrs

The metadata in a Zarr fileset is stored in (hidden) files starting with ".z".

In [15]:
find a.ome.zarr -name ".z*"

a.ome.zarr/OME/.zgroup
a.ome.zarr/OME/.zattrs
a.ome.zarr/.zgroup
a.ome.zarr/0/.zattrs
a.ome.zarr/0/1/.zarray
a.ome.zarr/0/0/.zarray
a.ome.zarr/0/.zgroup
a.ome.zarr/.zattrs


These are broken up into groups (folders) or arrays (data). The `.zgroup` files are fairly simple:

In [16]:
cat a.ome.zarr/.zgroup

{
  "zarr_format" : 2
}


Each `.zattrs` file contains user-supplied metadata. OME-Zarrs use these attributes to describe how an n-dimensional Zarr array should be interpreted as an image.

In [17]:
head a.ome.zarr/.zattrs a.ome.zarr/0/.zattrs


(firefox:51259): Gtk-WARNING **: 14:50:37.997: Locale not supported by C library.
	Using the fallback 'C' locale.
==> a.ome.zarr/.zattrs <==
{
  "bioformats2raw.layout" : 3
}
==> a.ome.zarr/0/.zattrs <==
{
  "multiscales" : [ {
    "metadata" : {
      "method" : "loci.common.image.SimpleImageScaler",
      "version" : "Bio-Formats 6.13.0"
    },
    "axes" : [ {
      "name" : "t",
      "type" : "time"
    }, {


The `.zarray` files specify details about storage like compression and array dimensions:

In [18]:
cat a.ome.zarr/0/0/.zarray

{
  "chunks" : [ 1, 1, 1, 512, 512 ],
  "compressor" : {
    "clevel" : 5,
    "blocksize" : 0,
    "shuffle" : 1,
    "cname" : "lz4",
    "id" : "blosc"
  },
  "dtype" : "|u1",
  "fill_value" : 0,
  "filters" : null,
  "order" : "C",
  "shape" : [ 1, 1, 1, 512, 512 ],
  "dimension_separator" : "/",
  "zarr_format" : 2
}


All the other files in the tree are **"chunks"**, pieces of an array that have been written to separate files:

In [19]:
tree a.ome.zarr

a.ome.zarr
|-- 0
|   |-- 0
|   |   `-- 0
|   |       `-- 0
|   |           `-- 0
|   |               `-- 0
|   |                   `-- 0
|   `-- 1
|       `-- 0
|           `-- 0
|               `-- 0
|                   `-- 0
|                       `-- 0
`-- OME
    `-- METADATA.ome.xml

12 directories, 3 files


The levels of this hierarchy can be interpreted as:
```
a.ome.zarr
└── series (Optional)
    └── resolution-level
        └── z-chunk-index
            └── y-chunk-index
                └── x-chunk-index
```

## 6d. Other options

In [20]:
bioformats2raw -h

Missing required parameter for option '--tile_height' (<tileHeight>)
Usage: <main class> [-p] [--keep-memo-files] [--no-hcs] [--[no-]minmax] [--[no-]
                    nested] [--no-ome-meta-export] [--no-root-group]
                    [--overwrite] [--use-existing-resolutions] [--version]
                    [--debug[=<logLevel>]] [--extra-readers[=<extraReaders>[,
                    <extraReaders>...]]]... [--options[=<readerOptions>[,
                    <readerOptions>...]]]... [-s[=<seriesList>[,
                    <seriesList>...]]]...
                    [--additional-scale-format-string-args=<additionalScaleForma
                    tCSV>] [-c=<compression>]
                    [--dimension-order=<dimensionOrder>]
                    [--downsample-type=<downsampling>]
                    [--fill-value=<fillValue>] [-h=<tileHeight>]
                    [--max_cached_tiles=<maxCachedTiles>]
                    [--max_workers=<maxWorkers>]
                    [--memo-director

In [21]:
ome_zarr info a.ome.zarr/0

/mnt/gluster2/scratch/josh_openmicroscopy/session1/a.ome.zarr/0 [zgroup]
 - metadata
   - Multiscales
   - OMERO
 - data
   - (1, 1, 1, 512, 512)
   - (1, 1, 1, 256, 256)


### Using MoBIE

mri.ome.zarr exported from ImageJ with the following macro:

```%java
run("MRI Stack");
Stack.setXUnit("mm"); // same unit for Y and Z
run("Properties...", "channels=1 slices=27 frames=1 pixel_width=1 pixel_height=1 voxel_depth=7");
run("Export Current Image To OME-ZARR...", "imagename=mri savedirectory=/tmp downsamplingmethod=Average usedefaults=true");
run("Open OME ZARR From File System...", "directory=/tmp/mri.ome.zarr");
```
<img src="mri.png" style="height:300px" />


## 6e. Extras (time-permitting)

### Renaming

Another important distinction to filesystems is that though it looks like hello is in a directory, you should really think of the entire string after the bucket just as a "key".

In [22]:
mc mv --recursive embo/bioimagecourse2023/session1/${YOURNAME}/a.ome.zarr/ embo/bioimagecourse2023/session1/${YOURNAME}/renamed.ome.zarr/

...A.ome.xml:  4.83 KiB / 4.83 KiB ┃▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓┃ 47.40 KiB/s 0s


Note the `/` endingings! This can cause a **lot** of headaches.

### Other resources

<table>
    <tr>
        <td>
            <a href="https://downloads.openmicroscopy.org/presentations/2020/Dundee/Workshops/NGFF/zarr_diagram/">
<img src="images/resources-1.png" alt="Screenshot of the Zarr diagram from OME2020" style="height:200px"/>
            </a>
        </td>
        <td>
<a href="https://downloads.openmicroscopy.org/presentations/2020/Dundee/Workshops/NGFF/zarr_diagram/">Diagram for how data moves</a>
        </td>
    </tr>
    <tr>
        <td>
      <a href="https://blog.openmicroscopy.org/file-formats/community/2020/11/04/zarr-data/">      
<img src="images/resources-2.png" alt="Screenshot of the Zarr diagram from OME2020" style="height:200px"/>
            </a>
        </td>
        <td>
<a href="https://blog.openmicroscopy.org/file-formats/community/2020/11/04/zarr-data/">Blog post for an easy way to publish OME-Zarr files</a>
        </td>
    </tr>
</table>    

6.4 Trying more with minio's play

Note, however, that play buckets get **deleted** every 24 hours.

In [23]:
mc mb --ignore-existing play/embo2023

Bucket created successfully `play/embo2023`.



In [24]:
mc policy set public play/embo2023

Access permission for `play/embo2023` is set to `public`



In [25]:
mc cp -r a.ome.zarr/ play/embo2023/a.ome.zarr/

...A.ome.xml:  4.83 KiB / 4.83 KiB ┃▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓┃ 3.69 KiB/s 1s


Now run: `napari https://play.minio.io/embo2023/a.ome.zarr`
<br/>
or checkout the validator: https://ome.github.io/ome-ngff-validator/?source=https://play.minio.io/embo2023/a.ome.zarr

### OMEZarrReader Jar

In [26]:
export BF_CP=/scratch/bioimagecourse2023/session1/OMEZarrReader-0.3.1-jar-with-dependencies.jar
showinf -nopix a.ome.zarr/OME/METADATA.ome.xml

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/mnt/gluster2/scratch/bioimagecourse2023/session1/mambaforge/envs/embo/share/bftools-7.0.0-0/bioformats_package.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/mnt/gluster2/scratch/bioimagecourse2023/session1/OMEZarrReader-0.3.1-jar-with-dependencies.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [ch.qos.logback.classic.util.ContextSelectorStaticBinder]
Checking file format [Zarr]
Initializing reader
ZarrReader initializing a.ome.zarr/OME/METADATA.ome.xml
Initialization took 1.879s

Reading core metadata
filename = a.ome.zarr/OME/METADATA.ome.xml
Used files:
	/mnt/gluster2/scratch/josh_openmicroscopy/session1/a.ome.zarr/OME/.zgroup
	/mnt/gluster2/scratch/josh_openmicroscopy/session1/a.ome.zarr/OME/.zattrs
	/mnt/gluster2/scratch/josh_openmicroscopy/session1/a

## License
Copyright (C) 2023 German BioImaging. All Rights Reserved.
This program is free software; you can redistribute it and/or modify it
under the terms of the GNU General Public License as published by the
Free Software Foundation; either version 2 of the License, or
(at your option) any later version.
This program is distributed in the hope that it will be useful, but
WITHOUT ANY WARRANTY; without even the implied warranty of MERCHANTABILITY
or FITNESS FOR A PARTICULAR PURPOSE. See the GNU General Public License for
more details. You should have received a copy of the GNU General
Public License along with this program; if not, write to the
Free Software Foundation,
Inc., 51 Franklin Street, Fifth Floor, Boston, MA 02110-1301 USA.